# Solar Locator and Analyzer - 805!

### Version:  0.1<br>Client:     Sunrise 805 (Davis Darnall)<br>Author:   Ryan Zinniger

Press Shift+Enter in the following cell to get started!

In [ ]:
import requests
import math
import numpy as np
import matplotlib.pyplot as plt

months = {"Str": ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"],
          "Int": [1,2,3,4,5,6,7,8,9,10,11,12]}

title = input("What is the title of this project?: ").title()

class Consumption():
    payload = {}
    def __init__(self):
        self.monthly = self.get_monthly()
        self.annual = self.get_annual(monthly=self.monthly)
        self.make_dict = self.write_dict(monthly=self.monthly, annual=self.annual)


    def get_monthly(self):
        print("Please enter the monthly kWh consumption for: ")
        self.monthly = []       
        for month in months["Str"]:
            strValue = input(f"\t{month}: ")
            intValue = int(strValue)
            self.monthly.append(intValue)
        return self.monthly

    def get_annual(self, monthly):
        self.annual = sum(monthly)
        return self.annual

    def write_dict(cls, monthly, annual):
        cls.payload["annual"] = annual
        cls.payload["monthly"] = monthly





class Production():
    payload = {}
    def __init__(self):
        self.mod_kwh = self.get_mod_kwh()

        print(f"\nPlease enter {title}'s address:\n\tCorrect Format: ### Street, City")
        self.address = input(f"\t{title}'s Address: ")
        
        self.normal_params = self.get_params(cap="1", address=self.address)
        self.normal_dict = self.pvw_req(pvw_params=self.normal_params)
        self.normal_monthly = self.get_monthly(dict=self.normal_dict)
        self.normal_annual = self.get_annual(monthly=self.normal_monthly)

        self.needed_kwh = float(Consumption.payload['annual']) / float(self.normal_annual)

        self.actual_params = self.get_params(cap=str(self.needed_kwh), address=self.address)
        self.actual_dict = self.pvw_req(pvw_params=self.actual_params)
        self.actual_monthly = self.get_monthly(dict=self.actual_dict)
        self.actual_annual = self.get_annual(monthly=self.actual_monthly)
        self.make_dict = self.write_dict(annual=self.actual_annual, monthly=self.actual_monthly,dc_size=self.needed_kwh, mod=self.mod_kwh)



    def get_mod_kwh(self):
        self.strModKWH = input("\nWhat is the production (kWh) for this project's modules?")
        self.intModKWH = float(self.strModKWH)
        return self.intModKWH

    def get_params(self, cap, address):
        self.capacity = cap
        self.azimuth = "180"
        self.tilt = "40"
        self.array_type = "1"
        self.module_type = "1"
        self.losses = "10"
        self.address = address

        pvw_inputs = {"system_capacity": self.capacity,
                      "azimuth": self.azimuth,
                      "tilt": self.tilt,
                      "array_type": self.array_type,
                      "module_type": self.module_type,
                      "losses": self.losses,
                      "address": self.address}
        return pvw_inputs

    def pvw_req(self, pvw_params):
        self.api_key = "95au0c4ycfTpXzbLTh5yHkaVX6P55qndz3XXUzg6"       #REPLACE WITH SUNRISE 805 KEY
        self.r = requests.get(f'https://developer.nrel.gov/api/pvwatts/v6.json?api_key={self.api_key}', params=pvw_params)
        self.dict = self.r.json()
        return self.dict

    def get_monthly(self, dict):
        self.ac_monthly = dict["outputs"]["ac_monthly"]
        self.ac_monthly_ = []
        for month in self.ac_monthly:
            round_month = int(month)
            self.ac_monthly_.append(round_month)
        return self.ac_monthly_

    def get_annual(self, monthly):
        self.ac_annual = sum(monthly)
        return round(self.ac_annual)

    def write_dict(cls, annual, monthly, dc_size, mod):
        cls.payload["annual"] = annual
        cls.payload["monthly"] = monthly
        cls.payload["dc_size"] = dc_size
        cls.payload["mod_kwh"] = mod



class Results():
    
    def __init__(self):
        self.annual_cons = Consumption.payload["annual"]
        self.monthly_cons = Consumption.payload["monthly"]
        self.annual_prod = Production.payload["annual"]
        self.monthly_prod = Production.payload["monthly"]
        self.needed_kwh = Production.payload["dc_size"]
        self.mod_kwh = Production.payload["mod_kwh"]
        self.graph_dict = {"Consumption":self.monthly_cons,
                           "Production":self.monthly_prod}

        print("\nResults:")
        self.mod_quantity = self.get_mod_quantity()
        self.savings = self.calculate_savings()
        self.graph = self.get_graph()

    def get_mod_quantity(self):
        self.mod_quantity = self.needed_kwh / self.mod_kwh
        self.mod_quantity_fig = math.ceil(self.mod_quantity)
        print(f"\t{title} will need {self.mod_quantity_fig} mods.")
        return math.ceil(self.mod_quantity)

    def calculate_savings(self):
        self.price_kwh = 0.28       #float(input("What is the cost (dollars) for a kWh? ")
        self.savings = self.annual_prod * self.price_kwh
        self.savings_fig = round(self.savings)
        print(f"\t{title}'s total annual savings will be: ${self.savings_fig}")
        return round(self.savings_fig)

    def get_graph(self):
        fig = plt.figure()
        fig, ax = plt.subplots()

        prod_y = np.array(self.graph_dict["Production"])
        cons_y = np.array(self.graph_dict["Consumption"])

        cons_graph = ax.bar(months["Str"], cons_y, width=1, color="orange",label="Consumption",edgecolor="black",capstyle='round')
        prod_graph = ax.plot(months["Str"], prod_y, "bo", label="Production", linestyle="--", linewidth=2, markersize=5)

        #ax.bar_label(cons_graph, padding=-15)
        ax.set_ylabel('kWh')
        ax.set_facecolor('C0')
        ax.set_title(f"{title}'s Consumption vs Production", fontsize="x-large")
        legend = ax.legend(fontsize="large")
        legend.get_frame().set_color("xkcd:salmon")
        plt.tight_layout()


        plt.savefig('testGraph')
        plt.show()
        pass

Consumption()
Production()
Results()